## Import modules

In [1]:
import pandas as pd
from datetime import datetime

## Read in and Inspect Data

In [23]:
moon_df = pd.read_csv('full-moon-calendar-1900-2050/full_moon.csv')
births1_df = pd.read_csv('fivethirtyeight-births-dataset/US_births_1994-2003_CDC_NCHS.csv')
births2_df = pd.read_csv('fivethirtyeight-births-dataset/US_births_2000-2014_SSA.csv')

print('moon_df:\n', moon_df.head(),
    '\n\nbirths1_df:\n', births1_df.head(),
    '\n\nbirths2_df:\n', births2_df.head())

moon_df:
          Day               Date          Time
0     Monday    15 January 1900   08:07:30 pm
1  Wednesday   14 February 1900   02:50:12 pm
2     Friday      16 March 1900   09:11:48 am
3     Sunday      15 April 1900   02:02:06 am
4     Monday        14 May 1900   04:36:36 pm 

births1_df:
    year  month  date_of_month  day_of_week  births
0  1994      1              1            6    8096
1  1994      1              2            7    7772
2  1994      1              3            1   10142
3  1994      1              4            2   11248
4  1994      1              5            3   11053 

births2_df:
    year  month  date_of_month  day_of_week  births
0  2000      1              1            6    9083
1  2000      1              2            7    8006
2  2000      1              3            1   11363
3  2000      1              4            2   13032
4  2000      1              5            3   12558


## Clean Data
- Check each dataframe for null values


In [24]:
print(
    'births1_df nulls:\n',
    births1_df[births1_df.isna().any(axis=1)],
    '\n\nbirths2_df nulls:\n',
    births2_df[births2_df.isna().any(axis=1)],
    '\n\nmoon_df nulls:\n',
    moon_df[moon_df.isna().any(axis=1)])

births1_df nulls:
 Empty DataFrame
Columns: [year, month, date_of_month, day_of_week, births]
Index: [] 

births2_df nulls:
 Empty DataFrame
Columns: [year, month, date_of_month, day_of_week, births]
Index: [] 

moon_df nulls:
 Empty DataFrame
Columns: [Day,  Date,  Time]
Index: []


- No null values found

- Check data types and eliminate any extra whitespace

In [25]:
# Check data types

print(
    'births1_df dtypes:\n', births1_df.dtypes, 
    '\n\nbirths2_df dtypes:\n', births2_df.dtypes,
    '\n\nmoon_df dtypes:\n', moon_df.dtypes)

births1_df dtypes:
 year             int64
month            int64
date_of_month    int64
day_of_week      int64
births           int64
dtype: object 

births2_df dtypes:
 year             int64
month            int64
date_of_month    int64
day_of_week      int64
births           int64
dtype: object 

moon_df dtypes:
 Day      object
 Date    object
 Time    object
dtype: object


In [ ]:
# The moon dataframe has strings, so str.strip can be applied

moon_df['Day'].str.strip()

In [ ]:
# Trying to strip the Date columns returned a keyerror for "Date", so inspecting columns names

moon_df.columns

In [ ]:
# Columns names Date and Time have extra whitespace, so need to remove that in order to operate on those columns

moon_df.columns = moon_df.columns.str.lstrip()
moon_df.columns

In [ ]:
# For date and time columns, we only want to remove any leading or trailing whitespace

moon_df['Date'].str.strip()

In [ ]:
moon_df['Time'].str.strip()

## Combining birth datasets

In [ ]:
# Combine birth data into one dataframe

births_df = pd.concat([births1_df, births2_df], ignore_index=True)

In [ ]:
# Confirm concatenation

births_df

- Drop duplicate dates since the datasets overlap from 2000-2003

In [ ]:
# Check the starting number of rows

births_df.shape

In [ ]:
# Drop rows that match on all three date components

births_df.drop_duplicates(subset=['year', 'month', 'date_of_month'], keep='last', inplace=True)

In [ ]:
# Verify new number of rows

births_df.shape

## Preparing birth data for merge with moon data
 
- For future use in Tableau, which recognizes date data, there should be a month column with the name of the month instead of a number.
- This will also make the month names the same as in moon_df

In [ ]:
# Create a dictionary of month numbers and names

month_name_dict = {
                1: 'January', 
                2: 'February', 
                3: 'March', 
                4: 'April', 
                5: 'May', 
                6:'June',
                7: 'July',
                8: 'August',
                9: 'September',
                10: 'October',
                11: 'November',
                12: 'December'
                }

In [ ]:
# Create series of month names so we can use series.map()

month_numbers = births_df['month']

In [ ]:
# Map month names to numbers

month_names = month_numbers.map(month_name_dict)

In [ ]:
# Add this series as a new column in births_df

births_df['month_name'] = month_names

In [ ]:
# Inspect results

births_df

- Do the same thing with day_of_week column to have day names as well as numbers, for possible births by day of week analysis in Tableau

In [ ]:
# According to kaggle documentation, 1 is Monday and 7 is Sunday

day_names_dict = {
                    1: 'Monday',
                    2: 'Tuesday',
                    3: 'Wednesday',
                    4: 'Thursday',
                    5: 'Friday',
                    6: 'Saturday',
                    7: 'Sunday'
                    }

In [ ]:
# Create series of day numbers from births_df

day_numbers = births_df['day_of_week']
day_numbers

In [ ]:
# Create day names series using series.map()

day_names = day_numbers.map(day_names_dict)

In [ ]:
# Adding day names series to births_df using same column name as moon_df

births_df['Day'] = day_names
births_df

## Preparing full moon dataset to merge with births

- Full moon dataset should have separate columns for month, day, and year, for easier merging with births

In [ ]:
moon_df.columns

In [ ]:
# Create date series from moon_df

moon_date = moon_df.loc[:, 'Date']
moon_date.head()

In [ ]:
# Split date into 3 columns

moon_date_df = moon_date.str.split(expand=True)
moon_date_df.head()

In [ ]:
# Renaming columns

moon_date_df.rename(columns={0: 'date_of_month', 1: 'month', 2: 'year'}, inplace=True)

In [ ]:
# Merging new columns back to moon_df

moon_df = moon_df.merge(moon_date_df, left_index=True, right_index=True)
moon_df